# Oil Price Prediction

## Constants

In [ ]:
BASE_PATH = 'BASE_PATH_OF_THE_PROJECT'

time: 1.37 ms (started: 2024-06-11 10:25:27 +00:00)


In [ ]:
CURRENT_PROJECT = 'check-clusterings'

time: 490 µs (started: 2024-06-11 10:25:27 +00:00)


## Necessary codes

In [ ]:
from google.colab import drive

drive.mount('/MyDrive', force_remount=True)

Mounted at /MyDrive
time: 4.24 s (started: 2024-06-11 10:25:27 +00:00)


In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 9.46 s (started: 2024-06-11 10:25:32 +00:00)


## Helper Functions

In [ ]:
def flatten(a):
  isChanged = None
  b = []
  while True:
    isChanged = False
    for x in a:
      try:
        b = b + list(x)
        isChanged = True
      except:
        b.append(x)
    a = b
    b = []
    if not isChanged:
      break
  return a

time: 964 µs (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def show_models_results(model, train_input, train_output, test_input, test_output, squeeze=True):
  def check_and_calc_squeeze(inp):
    if squeeze:
      return np.squeeze(inp)
    else:
      return inp

  print("train and test MAE: ",
        mean_absolute_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test RMSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0), squared=False),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0), squared=False))
  print("train and test MSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test MAPE: ",
        mean_absolute_percentage_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_percentage_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test R2: ",
        r2_score(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
      r2_score(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))


time: 1.41 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def show_models_results_with_labels(y_true, y_pred):
  print("MAE: ",
        mean_absolute_error(y_true, y_pred))
  print("RMSE: ",
        mean_squared_error(y_true, y_pred, squared=False))
  print("MSE: ",
        mean_squared_error(y_true, y_pred))
  print("MAPE: ",
        mean_absolute_percentage_error(y_true, y_pred))
  print("R2: ",
        r2_score(y_true, y_pred))


time: 1 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def write_models_and_cluster(models, cluster, model_name, trial_number):
  main_writing_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_writing_path + '/cluster'
  models_path = main_writing_path + "/models"

  os.makedirs(cluster_path)
  pickle.dump(cluster, open(cluster_path + '/cluster_file', 'wb'))

  for i, model in enumerate(models):
    current_model_path = models_path + "/" + str(i)
    os.makedirs(current_model_path)
    model.save(current_model_path)

time: 2.2 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def read_models_and_cluster(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  cluster_path = main_reading_path + '/cluster'
  models_path = main_reading_path + "/models"

  cluster = pickle.load(open(cluster_path + '/cluster_file', 'rb'))

  models = []

  for model in (os.listdir(models_path)):
    current_model_path = models_path + "/" + model
    models.append(tf.keras.models.load_model(current_model_path))

  return cluster, models

time: 9.58 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def is_model_trained(model_name, trial_number):
  main_reading_path = BASE_PATH + 'models/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  return os.path.exists(main_reading_path)

time: 4.62 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def write_model_results(model_name, trial_number, y_train_pred, y_test_pred):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  os.makedirs(main_writing_path)
  pd.DataFrame({'predictions': y_train_pred.squeeze()}).to_csv(main_writing_path + "/train.csv", index=False)
  pd.DataFrame({'predictions': y_test_pred.squeeze()}).to_csv(main_writing_path + "/test.csv", index=False)

time: 34.6 ms (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
def read_model_results(model_name, trial_number):
  main_writing_path = BASE_PATH + 'results/' + CURRENT_PROJECT + "/" + model_name + "/" + str(trial_number)
  train = pd.read_csv(main_writing_path + "/train.csv")
  test = pd.read_csv(main_writing_path + "/test.csv")
  return train, test

time: 837 µs (started: 2024-06-11 10:25:41 +00:00)


## Imports

In [ ]:
!pip install tslearn

time: 7.28 s (started: 2024-06-11 10:25:41 +00:00)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from sklearn.cluster import KMeans
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import random
from keras.constraints import Constraint
from keras import backend as kerasBackend
import seaborn as sns
import pickle
import os

time: 1.26 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
sns.set()

time: 4.57 ms (started: 2024-06-11 10:25:49 +00:00)


## Data Loading and Preparation

### Constants

In [ ]:
START_DATE = '01/04/2010'
END_DATE = '07/31/2020'
WTI_DATASET_PATH = BASE_PATH + 'data/processed-WTI-price-daily-investing.csv'
BRENT_DATASET_PATH = BASE_PATH + 'data/processed-Brent-price-daily-investing.csv'
TRAIN_PORTION = 0.7
VALIDATION_PORTION = 0
TEST_PORTION = 0.3
WINDOW_SIZE = 5

time: 936 µs (started: 2024-06-11 10:25:49 +00:00)


### Load data

In [ ]:
wti_whole = pd.read_csv(WTI_DATASET_PATH)
wti_whole

,Date,Price
0,01/04/2010,81.51
1,01/05/2010,81.77
2,01/06/2010,83.18
3,01/07/2010,82.66
4,01/08/2010,82.75
...,...,...
2741,07/27/2020,41.60
2742,07/28/2020,41.04
2743,07/29/2020,41.27
2744,07/30/2020,39.92


time: 85.7 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
brent_whole = pd.read_csv(BRENT_DATASET_PATH)
brent_whole

,Date,Price
0,01/04/2010,80.12
1,01/05/2010,80.59
2,01/06/2010,81.89
3,01/07/2010,81.51
4,01/08/2010,81.37
...,...,...
2728,07/27/2020,43.41
2729,07/28/2020,43.22
2730,07/29/2020,43.75
2731,07/30/2020,42.94


time: 97.6 ms (started: 2024-06-11 10:25:49 +00:00)


### Get Wanted Period

In [ ]:
wti_intersection = copy.deepcopy(wti_whole)
# wti_intersection = wti_whole[wti_whole["Date"] >= START_DATE]
# wti_intersection = wti_intersection[wti_intersection["Date"] <= END_DATE]
# assert (wti_intersection.iloc[0]["Date"] == START_DATE)
# assert (wti_intersection.iloc[-1]["Date"] == END_DATE)

time: 951 µs (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
brent_intersection = copy.deepcopy(brent_whole)
# brent_intersection = brent_whole[brent_whole["Date"] >= START_DATE]
# brent_intersection = brent_intersection[brent_intersection["Date"] <= END_DATE]
# assert (brent_intersection.iloc[0]["Date"] == START_DATE)
# assert (brent_intersection.iloc[-1]["Date"] == END_DATE)

time: 821 µs (started: 2024-06-11 10:25:49 +00:00)


### Get mean and standard deviation for normalization

In [ ]:
wti_mean = np.mean(wti_intersection['Price'])
wti_std = np.std(wti_intersection['Price'])
print("WTI mean and std: ", wti_mean, wti_std)

WTI mean and std:  70.20891478514203 22.941086577893046
time: 10.1 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
brent_mean = np.mean(brent_intersection['Price'])
brent_std = np.std(brent_intersection['Price'])
print("Brent mean and std: ", brent_mean, brent_std)

Brent mean and std:  77.96642151481889 26.59954316383742
time: 10.3 ms (started: 2024-06-11 10:25:49 +00:00)


### Divide into training and test

In [ ]:
number_of_datapoints_wti = len(wti_intersection)
number_of_datapoints_brent = len(brent_intersection)
number_of_datapoints_wti, number_of_datapoints_brent

(2746, 2733)

time: 8.65 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
train_part_wti = math.floor(number_of_datapoints_wti * TRAIN_PORTION)
validation_part_wti = math.floor(number_of_datapoints_wti * VALIDATION_PORTION)
test_part_wti = math.floor(number_of_datapoints_wti * TEST_PORTION)

time: 697 µs (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
train_part_brent = math.floor(number_of_datapoints_brent * TRAIN_PORTION)
validation_part_brent = math.floor(number_of_datapoints_brent * VALIDATION_PORTION)
test_part_brent = math.floor(number_of_datapoints_brent * TEST_PORTION)

time: 735 µs (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
wti_train = wti_intersection.iloc[:train_part_wti].reset_index(drop=True)
wti_validation = wti_intersection.iloc[train_part_wti:train_part_wti+validation_part_wti].reset_index(drop=True)
wti_test = wti_intersection.iloc[train_part_wti+validation_part_wti:].reset_index(drop=True)
print('number of records of parts: ', [len(wti_train), len(wti_validation), len(wti_test)])

number of records of parts:  [1922, 0, 824]
time: 10.3 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
brent_train = brent_intersection.iloc[:train_part_brent].reset_index(drop=True)
brent_validation = brent_intersection.iloc[train_part_brent:train_part_brent+validation_part_brent].reset_index(drop=True)
brent_test = brent_intersection.iloc[train_part_brent+validation_part_brent:].reset_index(drop=True)
print('number of records of parts: ', [len(brent_train), len(brent_validation), len(brent_test)])

number of records of parts:  [1913, 0, 820]
time: 12.2 ms (started: 2024-06-11 10:25:49 +00:00)


### Prepare inputs and targets

In [ ]:
import tensorflow as tf
import numpy as np
def get_input_and_output_tensor(df, feature_column, target_column, window_size):
  """
  inputs:
    df -> pandas dataframe
    feature_column -> column that is our feature
    target_column -> column that is our target
    window_size -> window size of inputs
  output:
    [inputs: (m ,window_size, 1), outputs: (m)]
  """
  inputs = np.zeros((len(df[feature_column]) - window_size, window_size, 1))
  outputs = np.zeros((len(df[feature_column]) - window_size))

  start = 0
  for i in range(window_size, len(df[feature_column])):
    inputs[start,:,0] = df[feature_column][start:i]
    outputs[start] = df[target_column][i]
    start = start + 1

  inputs = tf.convert_to_tensor(inputs)
  outputs = tf.convert_to_tensor(outputs)

  return [inputs, outputs]

time: 1.35 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
wti_train_inputs, wti_train_outputs = get_input_and_output_tensor(wti_train, "Price", "Price", WINDOW_SIZE)
if len(wti_validation) > 0:
  wti_validation_inputs, wti_validation_outputs = get_input_and_output_tensor(wti_validation, "Price", "Price", WINDOW_SIZE)
wti_test_inputs, wti_test_outputs = get_input_and_output_tensor(wti_test, "Price", "Price", WINDOW_SIZE)
print("train: ", wti_train_inputs.shape, wti_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, wti_validation_outputs.shape)
print("test: ", wti_test_inputs.shape, wti_test_outputs.shape)

train:  (1917, 5, 1) (1917,)
test:  (819, 5, 1) (819,)
time: 618 ms (started: 2024-06-11 10:25:49 +00:00)


In [ ]:
brent_train_inputs, brent_train_outputs = get_input_and_output_tensor(brent_train, "Price", "Price", WINDOW_SIZE)
if len(brent_validation) > 0:
  brent_validation_inputs, brent_validation_outputs = get_input_and_output_tensor(brent_validation, "Price", "Price", WINDOW_SIZE)
brent_test_inputs, brent_test_outputs = get_input_and_output_tensor(brent_test, "Price", "Price", WINDOW_SIZE)
print("train: ", brent_train_inputs.shape, brent_train_outputs.shape)
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, brent_validation_outputs.shape)
print("test: ", brent_test_inputs.shape, brent_test_outputs.shape)

train:  (1908, 5, 1) (1908,)
test:  (815, 5, 1) (815,)
time: 289 ms (started: 2024-06-11 10:25:50 +00:00)


### Normalize inputs

In [ ]:
wti_train_inputs = wti_train_inputs - wti_mean
wti_train_inputs = wti_train_inputs / wti_std

if len(wti_validation) > 0:
  wti_validation_inputs = wti_validation_inputs - wti_mean
  wti_validation_inputs = wti_validation_inputs / wti_std

wti_test_inputs = wti_test_inputs - wti_mean
wti_test_inputs = wti_test_inputs / wti_std

print("train: ", wti_train_inputs.shape, np.min(wti_train_inputs), np.max(wti_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", wti_validation_inputs.shape, np.min(wti_validation_inputs), np.max(wti_validation_inputs))
print("test: ", wti_test_inputs.shape, np.min(wti_test_inputs), np.max(wti_test_inputs))

train:  (1917, 5, 1) -1.917908928844772 1.9057983616604008
test:  (819, 5, 1) -4.700689063658387 0.270304773655908
time: 28.9 ms (started: 2024-06-11 10:25:50 +00:00)


In [ ]:
brent_train_inputs = brent_train_inputs - brent_mean
brent_train_inputs = brent_train_inputs / brent_std

if len(wti_validation) > 0:
  brent_validation_inputs = brent_validation_inputs - brent_mean
  brent_validation_inputs = brent_validation_inputs / brent_std

brent_test_inputs = brent_test_inputs - brent_mean
brent_test_inputs = brent_test_inputs / brent_std

print("train: ", brent_train_inputs.shape, np.min(brent_train_inputs), np.max(brent_train_inputs))
if len(wti_validation) > 0:
  print("validation: ", brent_validation_inputs.shape, np.min(brent_validation_inputs), np.max(brent_validation_inputs))
print("test: ", brent_test_inputs.shape, np.min(brent_test_inputs), np.max(brent_test_inputs))

train:  (1908, 5, 1) -1.8829805161057174 1.8302411505836444
test:  (815, 5, 1) -2.204414607937185 0.3129218586166238
time: 9.59 ms (started: 2024-06-11 10:25:50 +00:00)


## Experiments

### Compile and Fit parameters

In [ ]:
compile_parameters = {
    "optimizer": "adam",
    "loss": "mae"
}

fit_parameters = {
    "batch_size": 64,
    "epochs": 20,
    "shuffle": True,
    "verbose": 0,
}

time: 655 µs (started: 2024-06-11 10:25:50 +00:00)


### Constants

In [ ]:
NUMBER_OF_TRIALS = 10

time: 542 µs (started: 2024-06-11 10:25:50 +00:00)


### Kmeans-LSTM-Euclidean

#### Model Name

In [ ]:
MODEL_NAME = 'Kmeans_LSTM_Euclidean'

time: 482 µs (started: 2024-06-11 10:25:50 +00:00)


#### Main Clustering

In [ ]:
def cluster_data():
  KmeansWTI = TimeSeriesKMeans(n_clusters=WTI_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="euclidean",
                        verbose=False,
                        max_iter_barycenter=10,
                        random_state=0)

  KmeansWTI = KmeansWTI.fit(wti_train_inputs)

  KmeansBrent = TimeSeriesKMeans(n_clusters=BRENT_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="euclidean",
                        verbose=False,
                        max_iter_barycenter=10,
                        random_state=0)

  KmeansBrent = KmeansBrent.fit(brent_train_inputs)

  # WTI train
  wti_train_cluster_prediction = KmeansWTI.predict(wti_train_inputs)
  wti_train_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_train_cluster_prediction)):
    wti_train_clusters_inputs[wti_train_cluster_prediction[i]].append(wti_train_inputs[i])
    wti_train_clusters_outputs[wti_train_cluster_prediction[i]].append(wti_train_outputs[i])

  wti_train_clusters_inputs = [np.array(wti_train_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [np.array(wti_train_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # WTI test
  wti_test_cluster_prediction = KmeansWTI.predict(wti_test_inputs)
  wti_test_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_test_cluster_prediction)):
    wti_test_clusters_inputs[wti_test_cluster_prediction[i]].append(wti_test_inputs[i])
    wti_test_clusters_outputs[wti_test_cluster_prediction[i]].append(wti_test_outputs[i])

  wti_test_clusters_inputs = [np.array(wti_test_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [np.array(wti_test_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # Brent train
  brent_train_cluster_prediction = KmeansBrent.predict(brent_train_inputs)
  brent_train_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_train_cluster_prediction)):
    brent_train_clusters_inputs[brent_train_cluster_prediction[i]].append(brent_train_inputs[i])
    brent_train_clusters_outputs[brent_train_cluster_prediction[i]].append(brent_train_outputs[i])

  brent_train_clusters_inputs = [np.array(brent_train_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [np.array(brent_train_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  # Brent test
  brent_test_cluster_prediction = KmeansBrent.predict(brent_test_inputs)
  brent_test_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_test_cluster_prediction)):
    brent_test_clusters_inputs[brent_test_cluster_prediction[i]].append(brent_test_inputs[i])
    brent_test_clusters_outputs[brent_test_cluster_prediction[i]].append(brent_test_outputs[i])

  brent_test_clusters_inputs = [np.array(brent_test_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [np.array(brent_test_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  return (wti_train_clusters_inputs,
          wti_train_clusters_outputs,
          wti_test_clusters_inputs,
          wti_test_clusters_outputs,
          brent_train_clusters_inputs,
          brent_train_clusters_outputs,
          brent_test_clusters_inputs,
          brent_test_clusters_outputs,
          wti_train_cluster_prediction,
          wti_test_cluster_prediction,
          brent_train_cluster_prediction,
          brent_test_cluster_prediction,
          KmeansWTI,
          KmeansBrent,
  )

time: 2.75 ms (started: 2024-06-11 10:25:50 +00:00)


#### Make the main models and fit

In [ ]:
# best_wti_model = None
# best_wti_errors = [None, None]
# best_wti_history = None
# best_brent_model = None
# best_brent_errors = [None, None]
# best_brent_history = None

# fig, axis = plt.subplots(NUMBER_OF_TRIALS * 2, 3)

customized_fit_parameters = copy.deepcopy(fit_parameters)
customized_fit_parameters['epochs'] = 50

for number_of_clusters in range(2, 11, 2):
  BRENT_NUMBER_OF_CLUSTERS = number_of_clusters
  WTI_NUMBER_OF_CLUSTERS = number_of_clusters
  best_wti_models = None
  best_wti_error = None
  best_wti_cluster = None
  best_wti_predictions = None

  best_brent_models = None
  best_brent_error = None
  best_brent_cluster = None
  best_brent_predictions = None

  print('\n\ncurrent number of clusters: ', number_of_clusters)

  if (is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')):
    print('execution had been completed!')
    continue

  for _ in range(NUMBER_OF_TRIALS):
    print("start")

# ADDED
    if is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), _) or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), _):
      print('the model had been trained')

      brent_train_error, brent_test_error = read_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
      wti_train_error, wti_test_error = read_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), _)
      KmeansBrent, brent_models = read_models_and_cluster(MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
      KmeansWTI, wti_models = read_models_and_cluster(MODEL_NAME + "_WTI" + str(number_of_clusters), _)

      print('brent train with clustering: ')
      show_models_results_with_labels(brent_train_outputs, brent_train_error)
      print('brent test with clustering: ')
      show_models_results_with_labels(brent_test_outputs, brent_test_error)
      print("-"*20)
      print('wti train with clustering: ')
      show_models_results_with_labels(wti_train_outputs, wti_train_error)
      print('wti test with clustering: ')
      show_models_results_with_labels(wti_test_outputs, wti_test_error)

      wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_error)
      brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_error)

      if best_wti_models == None or wti_test_mae < best_wti_error:
        best_wti_models = wti_models
        best_wti_error = wti_test_mae
        best_wti_cluster = KmeansWTI
        best_wti_predictions = [wti_train_error, wti_test_error]

      if best_brent_models == None or brent_test_mae < best_brent_error:
        best_brent_models = brent_models
        best_brent_error = brent_test_mae
        best_brent_cluster = KmeansBrent
        best_brent_predictions = [brent_train_error, brent_test_error]

      print("END\n\n")
      continue


    (wti_train_clusters_inputs,
            wti_train_clusters_outputs,
            wti_test_clusters_inputs,
            wti_test_clusters_outputs,
            brent_train_clusters_inputs,
            brent_train_clusters_outputs,
            brent_test_clusters_inputs,
            brent_test_clusters_outputs,
            wti_train_cluster_prediction,
            wti_test_cluster_prediction,
            brent_train_cluster_prediction,
            brent_test_cluster_prediction,
            KmeansWTI,
            KmeansBrent,
    ) = cluster_data()
# END_ADDED


    wti_sizes = [random.randint(32, 512) for i in range(3)]
    brent_sizes = [random.randint(32, 512) for i in range(3)]

    # Dense
    wti_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])
    brent_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])

    wti_model.compile(**compile_parameters)
    brent_model.compile(**compile_parameters)

    wti_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
                              validation_data=((wti_test_inputs), wti_test_outputs))
    brent_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                              validation_data=((brent_test_inputs), brent_test_outputs))

    print("WTI Dense")
    show_models_results(wti_model, wti_train_inputs, wti_train_outputs,
                        wti_test_inputs, wti_test_outputs, squeeze=False)
    print("Brent Dense")
    show_models_results(brent_model, brent_train_inputs, brent_train_outputs,
                        brent_test_inputs, brent_test_outputs, squeeze=False)
    # sns.histplot(flatten(wti_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,0])
    # sns.histplot(flatten(brent_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,0])

    # Make each cluster's model and fine tune it
    wti_models = []
    brent_models = []

    for i in range(WTI_NUMBER_OF_CLUSTERS):
      model = tf.keras.models.clone_model(wti_model)
      changed_compile_parameters = copy.deepcopy(compile_parameters)
      changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
      model.compile(**changed_compile_parameters)
      model.set_weights(wti_model.get_weights())
      if len(wti_test_clusters_inputs[i]) == 0:
        model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters)
      else:
        model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters,
                                  validation_data=((wti_test_clusters_inputs[i]), wti_test_clusters_outputs[i]))
      wti_models.append(model)

    for i in range(BRENT_NUMBER_OF_CLUSTERS):
      model = tf.keras.models.clone_model(brent_model)
      changed_compile_parameters = copy.deepcopy(compile_parameters)
      changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
      model.compile(**changed_compile_parameters)
      model.set_weights(brent_model.get_weights())
      if len(brent_test_clusters_inputs[i]) == 0:
        model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters)
      else:
        model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters,
                                  validation_data=((brent_test_clusters_inputs[i]), brent_test_clusters_outputs[i]))
      brent_models.append(model)

# CHANGED

    # Evaluation
    wti_train_predicted_labels = np.zeros(wti_train_outputs.shape)
    wti_test_predicted_labels = np.zeros(wti_test_outputs.shape)

    for i in range(WTI_NUMBER_OF_CLUSTERS):

      if len(wti_test_clusters_outputs[i]) > 0:
        positions = np.where(wti_test_cluster_prediction == i)
        wti_test_predicted_labels[positions] = wti_models[i].predict((wti_test_clusters_inputs[i]), verbose=0).squeeze()

      if len(wti_train_clusters_outputs[i]) > 0:
        positions = np.where(wti_train_cluster_prediction == i)
        wti_train_predicted_labels[positions] = wti_models[i].predict((wti_train_clusters_inputs[i]), verbose=0).squeeze()

    brent_train_predicted_labels = np.zeros(brent_train_outputs.shape)
    brent_test_predicted_labels = np.zeros(brent_test_outputs.shape)

    for i in range(BRENT_NUMBER_OF_CLUSTERS):

      if len(brent_test_clusters_outputs[i]) > 0:
        positions = np.where(brent_test_cluster_prediction == i)
        brent_test_predicted_labels[positions] = brent_models[i].predict((brent_test_clusters_inputs[i]), verbose=0).squeeze()

      if len(brent_train_clusters_outputs[i]) > 0:
        positions = np.where(brent_train_cluster_prediction == i)
        brent_train_predicted_labels[positions] = brent_models[i].predict((brent_train_clusters_inputs[i]), verbose=0).squeeze()

    wti_train_mae = mean_absolute_error(wti_train_outputs, wti_train_predicted_labels)
    wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_predicted_labels)
    brent_train_mae = mean_absolute_error(brent_train_outputs, brent_train_predicted_labels)
    brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_predicted_labels)

    print("-"*20)
    print('wti train with clustering: ')
    show_models_results_with_labels(wti_train_outputs, wti_train_predicted_labels)
    print('wti test with clustering: ')
    show_models_results_with_labels(wti_test_outputs, wti_test_predicted_labels)
    print('brent train with clustering: ')
    show_models_results_with_labels(brent_train_outputs, brent_train_predicted_labels)
    print('brent test with clustering: ')
    show_models_results_with_labels(brent_test_outputs, brent_test_predicted_labels)
    print("END\n\n")
# END_CHANGED



# ADDED
    write_models_and_cluster(wti_models, KmeansWTI, MODEL_NAME + "_WTI" + str(number_of_clusters), _)
    write_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), _, wti_train_predicted_labels, wti_test_predicted_labels)
    write_models_and_cluster(brent_models, KmeansBrent, MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
    write_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), _, brent_train_predicted_labels, brent_test_predicted_labels)
# END_ADDED

    if best_wti_models == None or wti_test_mae < best_wti_error:
      best_wti_models = wti_models
      best_wti_error = wti_test_mae
      best_wti_cluster = KmeansWTI
      best_wti_predictions = [wti_train_predicted_labels, wti_test_predicted_labels]

    if best_brent_models == None or brent_test_mae < best_brent_error:
      best_brent_models = brent_models
      best_brent_error = brent_test_mae
      best_brent_cluster = KmeansBrent
      best_brent_predictions = [brent_train_predicted_labels, brent_test_predicted_labels]

  if not (is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')):
    write_models_and_cluster(best_wti_models, best_wti_cluster, MODEL_NAME + "_WTI" + str(number_of_clusters), 'best')
    write_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best', best_wti_predictions[0], best_wti_predictions[1])
    write_models_and_cluster(best_brent_models, best_brent_cluster, MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')
    write_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best', best_brent_predictions[0], best_brent_predictions[1])

# wti_model = best_wti_model
# brent_model = best_brent_model
# wti_model_history = best_wti_history
# brent_model_history = best_brent_history



current number of clusters:  2
execution had been completed!


current number of clusters:  4
execution had been completed!


current number of clusters:  6
execution had been completed!


current number of clusters:  8
execution had been completed!


current number of clusters:  10
execution had been completed!
time: 1.25 s (started: 2024-06-11 10:25:50 +00:00)


#### Show Results

In [ ]:
for number_of_clusters in range(2, 11, 2):
  wti_train, wti_test = read_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best')
  brent_train, brent_test = read_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')

  print("Number of clusters: ", number_of_clusters)
  print("WTI")
  show_models_results_with_labels(wti_test_outputs, wti_test)
  print("Brent")
  show_models_results_with_labels(brent_test_outputs, brent_test)
  print("-"*20)

Number of clusters:  2
WTI
MAE:  1.188545455373687
RMSE:  2.7083248033098006
MSE:  7.3350232402230695
MAPE:  0.02939189752352417
R2:  0.951323278152273
Brent
MAE:  1.000318969071277
RMSE:  1.4748729493575186
MSE:  2.1752502167465453
MAPE:  0.01960125880377003
R2:  0.9870721614597039
--------------------
Number of clusters:  4
WTI
MAE:  1.0834736372932552
RMSE:  2.557567503436262
MSE:  6.541151534633194
MAPE:  0.02517239059519048
R2:  0.9565915739613268
Brent
MAE:  1.0583942780289914
RMSE:  1.5216404615125743
MSE:  2.3153896941122003
MAPE:  0.020540014159069444
R2:  0.9862392915109702
--------------------
Number of clusters:  6
WTI
MAE:  1.1871165554340066
RMSE:  2.6485875181113214
MSE:  7.01501584109509
MAPE:  0.028086446170734553
R2:  0.9534469130265497
Brent
MAE:  1.0338956216449384
RMSE:  1.4943936552361907
MSE:  2.233212396810183
MAPE:  0.019953311947644516
R2:  0.9867276835235395
--------------------
Number of clusters:  8
WTI
MAE:  1.0457222446446306
RMSE:  2.558165993444216
MSE:

### Kmeans-LSTM-DTW

#### Model Name

In [ ]:
MODEL_NAME = 'Kmeans_LSTM_DTW'

time: 530 µs (started: 2024-06-11 10:25:58 +00:00)


#### Main Clustering

In [ ]:
def cluster_data():
  KmeansWTI = TimeSeriesKMeans(n_clusters=WTI_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="dtw",
                        verbose=False,
                        max_iter_barycenter=10,
                        random_state=0)

  KmeansWTI = KmeansWTI.fit(wti_train_inputs)

  KmeansBrent = TimeSeriesKMeans(n_clusters=BRENT_NUMBER_OF_CLUSTERS,
                        n_init=2,
                        metric="dtw",
                        verbose=False,
                        max_iter_barycenter=10,
                        random_state=0)

  KmeansBrent = KmeansBrent.fit(brent_train_inputs)

  # WTI train
  wti_train_cluster_prediction = KmeansWTI.predict(wti_train_inputs)
  wti_train_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_train_cluster_prediction)):
    wti_train_clusters_inputs[wti_train_cluster_prediction[i]].append(wti_train_inputs[i])
    wti_train_clusters_outputs[wti_train_cluster_prediction[i]].append(wti_train_outputs[i])

  wti_train_clusters_inputs = [np.array(wti_train_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_train_clusters_outputs = [np.array(wti_train_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # WTI test
  wti_test_cluster_prediction = KmeansWTI.predict(wti_test_inputs)
  wti_test_clusters_inputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [[] for i in range(WTI_NUMBER_OF_CLUSTERS)]

  for i in range(len(wti_test_cluster_prediction)):
    wti_test_clusters_inputs[wti_test_cluster_prediction[i]].append(wti_test_inputs[i])
    wti_test_clusters_outputs[wti_test_cluster_prediction[i]].append(wti_test_outputs[i])

  wti_test_clusters_inputs = [np.array(wti_test_clusters_inputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]
  wti_test_clusters_outputs = [np.array(wti_test_clusters_outputs[i]) for i in range(WTI_NUMBER_OF_CLUSTERS)]

  # Brent train
  brent_train_cluster_prediction = KmeansBrent.predict(brent_train_inputs)
  brent_train_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_train_cluster_prediction)):
    brent_train_clusters_inputs[brent_train_cluster_prediction[i]].append(brent_train_inputs[i])
    brent_train_clusters_outputs[brent_train_cluster_prediction[i]].append(brent_train_outputs[i])

  brent_train_clusters_inputs = [np.array(brent_train_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_train_clusters_outputs = [np.array(brent_train_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  # Brent test
  brent_test_cluster_prediction = KmeansBrent.predict(brent_test_inputs)
  brent_test_clusters_inputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [[] for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  for i in range(len(brent_test_cluster_prediction)):
    brent_test_clusters_inputs[brent_test_cluster_prediction[i]].append(brent_test_inputs[i])
    brent_test_clusters_outputs[brent_test_cluster_prediction[i]].append(brent_test_outputs[i])

  brent_test_clusters_inputs = [np.array(brent_test_clusters_inputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]
  brent_test_clusters_outputs = [np.array(brent_test_clusters_outputs[i]) for i in range(BRENT_NUMBER_OF_CLUSTERS)]

  return (wti_train_clusters_inputs,
          wti_train_clusters_outputs,
          wti_test_clusters_inputs,
          wti_test_clusters_outputs,
          brent_train_clusters_inputs,
          brent_train_clusters_outputs,
          brent_test_clusters_inputs,
          brent_test_clusters_outputs,
          wti_train_cluster_prediction,
          wti_test_cluster_prediction,
          brent_train_cluster_prediction,
          brent_test_cluster_prediction,
          KmeansWTI,
          KmeansBrent,
  )

time: 3.23 ms (started: 2024-06-11 10:25:58 +00:00)


#### Make the main models and fit

In [ ]:
# best_wti_model = None
# best_wti_errors = [None, None]
# best_wti_history = None
# best_brent_model = None
# best_brent_errors = [None, None]
# best_brent_history = None

# fig, axis = plt.subplots(NUMBER_OF_TRIALS * 2, 3)

customized_fit_parameters = copy.deepcopy(fit_parameters)
customized_fit_parameters['epochs'] = 50

for number_of_clusters in range(2, 11, 2):
  BRENT_NUMBER_OF_CLUSTERS = number_of_clusters
  WTI_NUMBER_OF_CLUSTERS = number_of_clusters
  best_wti_models = None
  best_wti_error = None
  best_wti_cluster = None
  best_wti_predictions = None

  best_brent_models = None
  best_brent_error = None
  best_brent_cluster = None
  best_brent_predictions = None

  print('\n\ncurrent number of clusters: ', number_of_clusters)

  if (is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')):
    print("this cluster number is done!")
    continue

  for _ in range(NUMBER_OF_TRIALS):
    print("start")

# ADDED
    if is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), _) or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), _):
      print('the model had been trained')

      brent_train_error, brent_test_error = read_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
      wti_train_error, wti_test_error = read_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), _)
      KmeansBrent, brent_models = read_models_and_cluster(MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
      KmeansWTI, wti_models = read_models_and_cluster(MODEL_NAME + "_WTI" + str(number_of_clusters), _)

      print('brent train with clustering: ')
      show_models_results_with_labels(brent_train_outputs, brent_train_error)
      print('brent test with clustering: ')
      show_models_results_with_labels(brent_test_outputs, brent_test_error)
      print("-"*20)
      print('wti train with clustering: ')
      show_models_results_with_labels(wti_train_outputs, wti_train_error)
      print('wti test with clustering: ')
      show_models_results_with_labels(wti_test_outputs, wti_test_error)

      wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_error)
      brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_error)

      if best_wti_models == None or wti_test_mae < best_wti_error:
        best_wti_models = wti_models
        best_wti_error = wti_test_mae
        best_wti_cluster = KmeansWTI
        best_wti_predictions = [wti_train_error, wti_test_error]

      if best_brent_models == None or brent_test_mae < best_brent_error:
        best_brent_models = brent_models
        best_brent_error = brent_test_mae
        best_brent_cluster = KmeansBrent
        best_brent_predictions = [brent_train_error, brent_test_error]

      print("END\n\n")
      continue


    (wti_train_clusters_inputs,
            wti_train_clusters_outputs,
            wti_test_clusters_inputs,
            wti_test_clusters_outputs,
            brent_train_clusters_inputs,
            brent_train_clusters_outputs,
            brent_test_clusters_inputs,
            brent_test_clusters_outputs,
            wti_train_cluster_prediction,
            wti_test_cluster_prediction,
            brent_train_cluster_prediction,
            brent_test_cluster_prediction,
            KmeansWTI,
            KmeansBrent,
    ) = cluster_data()
# END_ADDED


    wti_sizes = [random.randint(32, 512) for i in range(3)]
    brent_sizes = [random.randint(32, 512) for i in range(3)]

    # Dense
    wti_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(wti_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(wti_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(wti_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])
    brent_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(WINDOW_SIZE, 1), return_sequences=True),
        tf.keras.layers.LSTM(128),
        # tf.keras.layers.Dense(brent_sizes[0], 'leaky_relu'),
        # tf.keras.layers.Dense(brent_sizes[1], 'leaky_relu'),
        tf.keras.layers.Dense(brent_sizes[2], 'leaky_relu'),
        tf.keras.layers.Dense(1),
    ])

    wti_model.compile(**compile_parameters)
    brent_model.compile(**compile_parameters)

    wti_model.fit((wti_train_inputs), wti_train_outputs, **customized_fit_parameters,
                              validation_data=((wti_test_inputs), wti_test_outputs))
    brent_model.fit((brent_train_inputs), brent_train_outputs, **customized_fit_parameters,
                              validation_data=((brent_test_inputs), brent_test_outputs))

    print("WTI Dense")
    show_models_results(wti_model, wti_train_inputs, wti_train_outputs,
                        wti_test_inputs, wti_test_outputs, squeeze=False)
    print("Brent Dense")
    show_models_results(brent_model, brent_train_inputs, brent_train_outputs,
                        brent_test_inputs, brent_test_outputs, squeeze=False)
    # sns.histplot(flatten(wti_model.get_weights()), bins=(80), kde=True, ax=axis[_*2,0])
    # sns.histplot(flatten(brent_model.get_weights()), bins=(80), kde=True, ax=axis[_*2+1,0])

    # Make each cluster's model and fine tune it
    wti_models = []
    brent_models = []

    for i in range(WTI_NUMBER_OF_CLUSTERS):
      model = tf.keras.models.clone_model(wti_model)
      changed_compile_parameters = copy.deepcopy(compile_parameters)
      changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
      model.compile(**changed_compile_parameters)
      model.set_weights(wti_model.get_weights())
      if len(wti_test_clusters_inputs[i]) == 0:
        model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters)
      else:
        model_history = model.fit((wti_train_clusters_inputs[i]), wti_train_clusters_outputs[i], **customized_fit_parameters,
                                  validation_data=((wti_test_clusters_inputs[i]), wti_test_clusters_outputs[i]))
      wti_models.append(model)

    for i in range(BRENT_NUMBER_OF_CLUSTERS):
      model = tf.keras.models.clone_model(brent_model)
      changed_compile_parameters = copy.deepcopy(compile_parameters)
      changed_compile_parameters["optimizer"] = tf.keras.optimizers.Adam(0.0001)
      model.compile(**changed_compile_parameters)
      model.set_weights(brent_model.get_weights())
      if len(brent_test_clusters_inputs[i]) == 0:
        model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters)
      else:
        model_history = model.fit((brent_train_clusters_inputs[i]), brent_train_clusters_outputs[i], **customized_fit_parameters,
                                  validation_data=((brent_test_clusters_inputs[i]), brent_test_clusters_outputs[i]))
      brent_models.append(model)

# CHANGED

    # Evaluation
    wti_train_predicted_labels = np.zeros(wti_train_outputs.shape)
    wti_test_predicted_labels = np.zeros(wti_test_outputs.shape)

    for i in range(WTI_NUMBER_OF_CLUSTERS):

      if len(wti_test_clusters_outputs[i]) > 0:
        positions = np.where(wti_test_cluster_prediction == i)
        wti_test_predicted_labels[positions] = wti_models[i].predict((wti_test_clusters_inputs[i]), verbose=0).squeeze()

      if len(wti_train_clusters_outputs[i]) > 0:
        positions = np.where(wti_train_cluster_prediction == i)
        wti_train_predicted_labels[positions] = wti_models[i].predict((wti_train_clusters_inputs[i]), verbose=0).squeeze()

    brent_train_predicted_labels = np.zeros(brent_train_outputs.shape)
    brent_test_predicted_labels = np.zeros(brent_test_outputs.shape)

    for i in range(BRENT_NUMBER_OF_CLUSTERS):

      if len(brent_test_clusters_outputs[i]) > 0:
        positions = np.where(brent_test_cluster_prediction == i)
        brent_test_predicted_labels[positions] = brent_models[i].predict((brent_test_clusters_inputs[i]), verbose=0).squeeze()

      if len(brent_train_clusters_outputs[i]) > 0:
        positions = np.where(brent_train_cluster_prediction == i)
        brent_train_predicted_labels[positions] = brent_models[i].predict((brent_train_clusters_inputs[i]), verbose=0).squeeze()

    wti_train_mae = mean_absolute_error(wti_train_outputs, wti_train_predicted_labels)
    wti_test_mae = mean_absolute_error(wti_test_outputs, wti_test_predicted_labels)
    brent_train_mae = mean_absolute_error(brent_train_outputs, brent_train_predicted_labels)
    brent_test_mae = mean_absolute_error(brent_test_outputs, brent_test_predicted_labels)

    print("-"*20)
    print('wti train with clustering: ')
    show_models_results_with_labels(wti_train_outputs, wti_train_predicted_labels)
    print('wti test with clustering: ')
    show_models_results_with_labels(wti_test_outputs, wti_test_predicted_labels)
    print('brent train with clustering: ')
    show_models_results_with_labels(brent_train_outputs, brent_train_predicted_labels)
    print('brent test with clustering: ')
    show_models_results_with_labels(brent_test_outputs, brent_test_predicted_labels)
    print("END\n\n")
# END_CHANGED



# ADDED
    write_models_and_cluster(wti_models, KmeansWTI, MODEL_NAME + "_WTI" + str(number_of_clusters), _)
    write_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), _, wti_train_predicted_labels, wti_test_predicted_labels)
    write_models_and_cluster(brent_models, KmeansBrent, MODEL_NAME + "_BRENT" + str(number_of_clusters), _)
    write_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), _, brent_train_predicted_labels, brent_test_predicted_labels)
# END_ADDED

    if best_wti_models == None or wti_test_mae < best_wti_error:
      best_wti_models = wti_models
      best_wti_error = wti_test_mae
      best_wti_cluster = KmeansWTI
      best_wti_predictions = [wti_train_predicted_labels, wti_test_predicted_labels]

    if best_brent_models == None or brent_test_mae < best_brent_error:
      best_brent_models = brent_models
      best_brent_error = brent_test_mae
      best_brent_cluster = KmeansBrent
      best_brent_predictions = [brent_train_predicted_labels, brent_test_predicted_labels]

  if not (is_model_trained(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best') or is_model_trained(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')):
    write_models_and_cluster(best_wti_models, best_wti_cluster, MODEL_NAME + "_WTI" + str(number_of_clusters), 'best')
    write_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best', best_wti_predictions[0], best_wti_predictions[1])
    write_models_and_cluster(best_brent_models, best_brent_cluster, MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')
    write_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best', best_brent_predictions[0], best_brent_predictions[1])

# wti_model = best_wti_model
# brent_model = best_brent_model
# wti_model_history = best_wti_history
# brent_model_history = best_brent_history



current number of clusters:  2
this cluster number is done!


current number of clusters:  4
this cluster number is done!


current number of clusters:  6
this cluster number is done!


current number of clusters:  8
this cluster number is done!


current number of clusters:  10
this cluster number is done!
time: 737 ms (started: 2024-06-11 10:25:58 +00:00)


#### Show Results

In [ ]:
for number_of_clusters in range(2, 11, 2):
  wti_train, wti_test = read_model_results(MODEL_NAME + "_WTI" + str(number_of_clusters), 'best')
  brent_train, brent_test = read_model_results(MODEL_NAME + "_BRENT" + str(number_of_clusters), 'best')

  print("Number of clusters: ", number_of_clusters)
  print("WTI")
  show_models_results_with_labels(wti_test_outputs, wti_test)
  print("Brent")
  show_models_results_with_labels(brent_test_outputs, brent_test)
  print("-"*20)

Number of clusters:  2
WTI
MAE:  1.1073424872839694
RMSE:  2.575007419121297
MSE:  6.630663208529723
MAPE:  0.026199460160824185
R2:  0.9559975560953039
Brent
MAE:  1.0079570543722138
RMSE:  1.4667302188058675
MSE:  2.1512975347583083
MAPE:  0.019493424057239812
R2:  0.9872145158440258
--------------------
Number of clusters:  4
WTI
MAE:  1.1051174595591784
RMSE:  2.5792199241941467
MSE:  6.6523754173600596
MAPE:  0.025709194446169804
R2:  0.955853469414823
Brent
MAE:  1.097625831838034
RMSE:  1.537301960659166
MSE:  2.363297318246516
MAPE:  0.02094018569235801
R2:  0.9859545693098691
--------------------
Number of clusters:  6
WTI
MAE:  1.2390635212582688
RMSE:  2.7839124080577786
MSE:  7.750168295738059
MAPE:  0.03109238131082444
R2:  0.9485682902358138
Brent
MAE:  1.0229467243592432
RMSE:  1.5020521582079358
MSE:  2.2561606859771177
MAPE:  0.020421370872037728
R2:  0.9865912983965126
--------------------
Number of clusters:  8
WTI
MAE:  1.1164830701779094
RMSE:  2.598731396894119
MS